In [ ]:
import os
import numpy as np
from databaseemulator import DataBaseEmulator
from domain_model import StateVariable, Activity

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_scenarios", "cut_in_scenarios.json")
cutins = DataBaseEmulator(filename)
nscenarios = len(cutins.collections["scenario"])
print("Number of scenarios: {:d}".format(nscenarios))

In [ ]:
# Go through all cut-ins and store the sequence of longitudinal activities.
activities = []
for i in range(nscenarios):
    # Get the acts in chronological order.
    scenario = cutins.get_item("scenario", i)
    acts = [scenario.acts[i] for i in np.argsort([act[2] for act in scenario.acts])]
    
    # List the longitudinal activities of the target vehicle.
    curr_activities = []
    for act in acts:
        if act[0].name == "target vehicle" and \
                act[1].activity_category.state == StateVariable.LON_TARGET:
            curr_activities.append(act[1].name[0])
    activities.append(curr_activities)

In [ ]:
# Store in a dictionary the possibilities of 'item' sequences.
# 'af' means "accelerating and lane following"
# 'dl' means "decelerating and lane changing"
# 'cf', 'df', 'al', and 'cl' are similarly defined.
def get_transition(activities, main_act, other_acts):
    counter = np.zeros(1+len(other_acts), dtype=np.int)
    for activity in activities:
        try:
            i = activity.index(main_act)
        except ValueError:
            pass
        else:
            if i == len(activity)-1:
                counter[len(other_acts)] += 1
                continue
            for j, other_act in enumerate(other_acts):
                if activity[i+1] == other_act:
                    counter[j] += 1
    counter = counter / np.sum(counter)
    probs = dict()
    probs['end'] = counter[-1]
    for i, other_act in enumerate(other_acts):
        probs[other_act+'l'] = counter[i]
    return probs
transition = dict(af=dict(al=1),
                  cf=dict(cl=1),
                  df=dict(dl=1),
                  al=get_transition(activities, 'a', ['c', 'd']),
                  cl=get_transition(activities, 'c', ['a', 'd']),
                  dl=get_transition(activities, 'd', ['a', 'c']))

In [ ]:
acts, counts = np.unique([activity[0] for activity in activities], return_counts=True)
transition['start'] = dict()
for act, count in zip(acts, counts/np.sum(counts)):
    transition['start'][act+'f'] = count
transition

In [ ]:
def next_item(probs):
    """ It is important that the probabilities in probs add up to 1. """
    rand = np.random.rand()
    for key, value in probs.items():
        if rand <= value:
            return key
        rand -= value

def generate_sequence(transitions, seed=None):
    if seed is not None:
        np.random.seed(0)
    sequence = ['start']
    while not sequence[-1] == 'end':
        sequence.append(next_item(transitions[sequence[-1]]))
    return sequence[1:-1]

def probability_sequence(transitions, sequence):
    sequence = ['start'] + sequence + ['end']
    prob = 1
    for act1, act2 in zip(sequence[:-1], sequence[1:]):
        prob *= transitions[act1][act2]
    return prob

In [ ]:
np.random.seed(0)
n = 10000
seqs = [generate_sequence(transition) for _ in range(n)]

In [ ]:
sequences = [['cf', 'cl'],
             ['af', 'al'],
             ['df', 'dl'],
             ['cf', 'cl', 'al'],
             ['cf', 'cl', 'dl'],
             ['af', 'al', 'cl'],
             ['af', 'al', 'dl'],
             ['df', 'dl', 'al'],
             ['df', 'dl', 'cl']] #,
x=[             ['cf', 'cl', 'al', 'cl'],
             ['cf', 'cl', 'al', 'dl'],
             ['cf', 'cl', 'dl', 'cl'],
             ['cf', 'cl', 'dl', 'al'],
             ['af', 'al', 'cl', 'al'],
             ['af', 'al', 'cl', 'dl'],
             ['af', 'al', 'dl', 'al'],
             ['af', 'al', 'dl', 'cl'],
             ['df', 'dl', 'al', 'cl'],
             ['df', 'dl', 'al', 'dl'],
             ['df', 'dl', 'cl', 'al'],
             ['df', 'dl', 'cl', 'dl']]
probs = [probability_sequence(transition, seq) for seq in sequences]
# .count() is very slow like this, but fine for now...
mc_probs = [seqs.count(seq)/n for seq in sequences]
real_seqs = [[y[0]+'f']+[x+'l' for x in y] for y in activities]
real_probs = [real_seqs.count(seq)/len(real_seqs) for seq in sequences]
for seq, prob, mc_prob, real_prob in zip(sequences, probs, mc_probs, real_probs):
    print("{:13s} {:.4f} {:.4f} {:.4f}".format(' '.join(seq), prob, mc_prob, real_prob))
print()
print("{:13s} {:.4f} {:.4f} {:.4f}".format("Total", np.sum(probs), np.sum(mc_probs), 
                                           np.sum(real_probs)))

In [ ]:
# Get the parameters of the different activities.
def parm_acc_dec(activity: Activity):
    """ Get initial speed, mean acceleration, and speed difference. """
    vstart, vend = activity.get_state(time=[activity.tstart, activity.tend])[0]
    amean = (vend - vstart) / (activity.tend - activity.tstart)
    return vstart, amean, vend-vstart

def parm_cruise(activity: Activity):
    """ Get initial speed, speed difference"""

In [ ]:
a = cutins.get_item("activity", 1)

In [ ]:
vstart, vend = a.get_state(time=[a.tstart, a.tend])[0]

In [ ]:
parm_acc_dec(cutins.get_item("activity", 1))

In [ ]:
vend